In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import Sequence , get_file
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler ,EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf


import matplotlib.pyplot as plt
import numpy as np
import cv2
import os, json, shutil
import os
import json
import sys
from datetime import datetime
import os
import time
from itertools import islice

import segmentation_models as sm
sm.set_framework('tf.keras')


# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")    
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)
    
# tf.config.set_soft_device_placement(True)

mirrored_strategy = tf.distribute.MirroredStrategy()


class DataGenerator(Sequence):
    def __init__(self, list_IDs,label_map , img_dir ,mode):
        'Initialization'
        self.list_IDs = list_IDs
        self.label_map = image_label_map
        self.on_epoch_end()
        self.img_dir = img_dir + "/images"
        self.mask_dir = img_dir + "/masks"
        self.mode = mode

    def __len__(self):
        return int(len(self.list_IDs))

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))    

    
    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index:(index+1)]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
    
    def __data_generation(self, list_IDs_temp):
        """Generates data containing batch_size samples"""
        if self.mode == "train":
            # Generate data
            X, y = self.load_file(list_IDs_temp)
            return X, y
        elif self.mode == "val":
            X, y = self.load_file(list_IDs_temp)
            return X, y
        
        
    def load_file(self, id_list):
        list_IDs_temp = id_list
        for ID in list_IDs_temp:
            x_file_path = os.path.join(self.img_dir, ID)
            y_file_path = os.path.join(self.mask_dir, self.label_map.get(ID))
            # Store sample
            X = np.load(x_file_path)
            # Store class
            y = np.load(y_file_path).astype('float32')
        return X, y    
    

    
    
out_train_data_dir = '/glade/scratch/hgamarro/Deep_learning/HG_space/data/processed/Vegas_v1/train'
out_val_data_dir = '/glade/scratch/hgamarro/Deep_learning/HG_space/data/processed/Vegas_v1/val'


# ====================
# train set
# ====================
all_files = [s for s in os.listdir(out_train_data_dir + "/images/") if s.endswith('.npy')]
all_files.append([s for s in os.listdir(out_train_data_dir + "/masks/") if s.endswith('.npy')] )

image_label_map = {
        "image_file_{}.npy".format(i+1): "label_file_{}.npy".format(i+1)
        for i in range(int(len(all_files)))}
partition = [item for item in all_files if "image_file" in item]

# ====================
# validation set
# ====================

all_val_files = [s for s in os.listdir(out_val_data_dir + "/images/") if s.endswith('.npy')]
all_val_files.append([s for s in os.listdir(out_val_data_dir + "/masks/") if s.endswith('.npy')] )
val_image_label_map = {
        "image_file_{}.npy".format(i+1): "label_file_{}.npy".format(i+1)
        for i in range(int(len(all_val_files)))}
val_partition = [item for item in all_val_files if "image_file" in item]


train_generator = DataGenerator(partition,image_label_map,out_train_data_dir, "train")
val_generator= DataGenerator(val_partition,val_image_label_map,out_val_data_dir, "val")

Segmentation Models: using `tf.keras` framework.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


In [2]:
save_path = "/glade/scratch/hgamarro/Deep_learning/HG_space/models/scratch/BD_v1_results_main/"

In [8]:
BACKBONE = 'vgg16'
# BACKBONE = 'resnet50'
# BACKBONE = 'efficientnetb6'
model_name = 'Unet'
BATCH_SIZE = 32
LR = 0.0001
EPOCHS = 100

n_classes = 1  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model

with mirrored_strategy.scope():
    if model_name == 'Unet':
        print(model_name)
        model = sm.Unet(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
    elif model_name == 'Linknet':
        print(model_name)
        model = sm.Linknet(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
    elif model_name == 'FPN':
        print(model_name)
        model = sm.FPN(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
    

# define optomizer
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint(save_path + BACKBONE + "_" + model_name + ".h5",
                                    monitor='val_loss', 
                                    save_best_only=True, 
                                    mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
    keras.callbacks.EarlyStopping(monitor='val_loss',
                           patience=5,
                           restore_best_weights=True,
                           mode='min')
]


Unet


In [9]:
start = datetime.now()

print("Model Fit")


history = model.fit(
    train_generator, 
    steps_per_epoch=len(train_generator), 
    epochs=EPOCHS, 
    callbacks=callbacks, 
    validation_data=val_generator, 
    validation_steps=len(val_generator)
)


end = datetime.now()
print("\nTime Taken for testing: %s" % (end-start))

Model Fit
Epoch 1/100
INFO:tensorflow:batch_all_reduce: 64 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 64 all-reduces with algorithm = nccl, num_packs = 1
35/35 [==============================] - 61s 2s/step - loss: 0.7842 - iou_score: 0.3400 - f1-score: 0.4984 - val_loss: 1.8003 - val_iou_score: 0.2058 - val_f1-score: 0.3410
Epoch 2/100
35/35 [==============================] - 58s 2s/step - loss: 0.5850 - iou_score: 0.6433 - f1-score: 0.7817 - val_loss: 1.0188 - val_iou_score: 0.2753 - val_f1-score: 0.4313
Epoch 3/100
35/35 [==============================] - 59s 2s/step - loss: 0.5033 - iou_score: 0.7487 - f1-score: 0.8560 - val_loss: 0.6385 - val_iou_score: 0.4985 - val_f1-score: 0.6645
Epoch 4/100
35/35 [==============================] - 58s 2s/step - loss: 0.4546 - iou_score: 0.7776 - f1-score: 0.8747 - val_loss: 0.5048 - val_iou_score: 0.7394 - val_f1-score: 0.8500
Epoch 5/100
35/35 [==============================] - 59s 2s/step - loss: 0.405

In [ ]:
tf.keras.backend.clear_session()


BACKBONE = 'vgg16'
# BACKBONE = 'resnet50'
# BACKBONE = 'efficientnetb6'
# model_name = 'Unet'
model_name = 'Linknet'
BATCH_SIZE = 32
LR = 0.0001
EPOCHS = 50

n_classes = 1  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model

with mirrored_strategy.scope():
    if model_name == 'Unet':
        print(model_name)
        model = sm.Unet(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
    elif model_name == 'Linknet':
        print(model_name)
        model = sm.Linknet(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
    elif model_name == 'FPN':
        print(model_name)
        model = sm.FPN(BACKBONE,classes=n_classes,input_shape=(512, 512, 3), activation=activation)
    

# define optomizer
optim = keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint(save_path + BACKBONE + "_" + model_name + ".h5",
                                    monitor='val_loss', 
                                    save_best_only=True, 
                                    mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
    keras.callbacks.EarlyStopping(monitor='val_loss',
                           patience=5,
                           restore_best_weights=True,
                           mode='min')
]



In [ ]:
start = datetime.now()

print("Model Fit")


history = model.fit(
    train_generator, 
    steps_per_epoch=len(train_generator), 
    epochs=EPOCHS, 
    callbacks=callbacks, 
    validation_data=val_generator, 
    validation_steps=len(val_generator)
)


end = datetime.now()
print("\nTime Taken for testing: %s" % (end-start))